## MapReduce WordCount en Java

Comme présenté dans le cours, Hadoop MapReduce a été initialement développé en Java et pour cette raison utilise préférentiellement du code source en Java. 

Dans ce premier exercice, vous allez apprendre à écrire et compiler un code pour compter les mots des livres de notre dataset.



### Etape 1 : créer un "Driver"

On appelle **driver** la classe qui est appellée en premier lors de l'exécution d'un programme. Dans notre cas, cette classe aura le rôle d'initialiser l'environnement Hadoop et de coordonner qui sera appelé à chaque étape (Map, Reduce). 

Dans le paragraphe suivant vous pouvez voir le code source pour un "driver". 

Cette classe (qui appartient au package "urca.bigdata") se connecte à l'environnement Hadoop en créant un `Job` "**word count**". Par la suite, elle renseigne certaines informations utiles au déroulement du programme : 
* quelle sera la classe utilisée pour le map --> `job.setMapperClass(TokenizerMapper.class);`
* quelle sera la classe utilisée pour le reduce --> `job.setReducerClass(IntSumReducer.class);`
* quelle sera la classe utilisée pour le combiner (optimisation locale) --> `job.setCombinerClass(IntSumReducer.class);`
* quel est le format de la clé produite à la sortie (texte) --> `job.setOutputKeyClass(Text.class);`
* quel est le format de la valeur produite à la sortie (entier) --> `job.setOutputValueClass(IntWritable.class);`
* le point d'entrée pour la lecture des données à traiter (chemin spécifié dans le premier argument d'appel) --> `FileInputFormat.addInputPath(job, new Path(args[0]));`
* l'endroi où les résultats seront enregistrés (deuxième argument d'appel) --> `FileOutputFormat.setOutputPath(job, new Path(args[1]));`

On voit donc que ce **driver** fait appel à d'autres classes. Regardons tout d'abord la classe *TokenizerMapper.java* :

Cette classe reçoit de Hadoop une ligne de texte avec une clé en format *Object*. Comme nous voulons compter les mots, la méthode `map()` effectue une *tokenisation*, c'est à dire, elle découpe la ligne en mots. 

Pour chaque mot produit, une paire **clé, valeur** sera produite (`context.write(word, one)`):
* *word* contient le mot trouvé
* *one* est un entier **1**

L'ensemble des ces clés/valeurs passera d'abord par une optimisation locale (*combiner*) qui regroupera tous les mots identiques dans une machine, puis sera transmis au *Reducer* pour produire le comptage final. Comme l'opération combiner et reducer est la même (la seule chose qui change est l'étendu de l'opération) on utilise la même classe `IntSumReducer.java` :


Ici, la méthode `reduce()` reçoit des ensembles {clé, {liste de valeurs}}. En parcourant la liste, cette méthode récupère les valeurs et les additione, afin de produire une clé finale contenant **mot, somme**. 

## Compiler et exécuter

C'est le moment de mettre tout ensemble. 
 Tout d'abord, vous allez compléter le code de chacun des fichiers avec le code indiqué ci-dessus. Pour cela, ouvrez chacun de ces fichiers à partir de la racine du dashboard (vous pouvez y retourner en cliquant sur le logo "jupyterhub" en haut à gauche), et copiez le code des paragraphes respectifs :
 
* urca/bigdata/WordCount.java
* urca/bigdata/TokenizerMapper.java
* urca/bigdata/IntSumReducer.java


### Compiler

C'est bon, les fichiers sont remplis et sauvegardés ? C'est l'heure de les compiler et produire un **jar** pour Hadoop
* si vous voulez, vous pouvez ouvrir un terminal à partir de la racine du dashboard (new->Terminal) et exécuter ces commandes vous même.

In [ ]:
## mise en place d'un répertoire pour le code compilé
! export HADOOP_CLASSPATH=$(hadoop classpath)
! mkdir WordCount


In [ ]:
## mise en place des variables d'environnement et compilation
! javac -classpath `hadoop classpath`:. -d WordCount urca/bigdata/WordCount.java


In [ ]:
## création du jar
! jar -cvf WC.jar -C WordCount/ .

# Exécution

Voilà, tout est prêt. On a un Jar avec les exécutables qui Hadoop peut utiliser pour déployer l'application. On a le dataset dans HDFS. Il ne reste qu'à lancer le code.

Pour cela, on va appeler Hadoop avec certains paramètres que vous devez pouvoir comprendre facilement : 

`hadoop jar WC.jar urca.bigdata.WordCount livres countLivres`

On dit à hadoop de lancer le jar *WC.jar* et d'appeler la classe *urca.bigdata.WordCount*.
Ensuite, on passe comme paramètres deux répertoires de HDFS :
* le répertoire *livres* qui contient notre dataset
* le répertoire *countLivres* qui contiendra le résultat

### ATTENTION : le répertoire pour les résultats ne doit pas être crée avant : Hadoop se refuse d'écraser un répertoire existant 

Quand vous lancerez la commande dans le paragraphe ci-dessous, vous verez la progression du calcul. 


In [ ]:
! hadoop jar WC.jar urca.bigdata.WordCount livres countLivres

Pour finir, on peut regarder le résultat final stocké dans HDFS (et même le rappatrier) :

In [ ]:
! hdfs dfs -ls countLivres

In [ ]:
! hdfs dfs -cat countLivres/part-r-00000 | tail -200

On affiche une petite partie du résultat, mais vous pouvez observer qu'il y a encore du travail à faire pour rendre un résultat propre. Par exemple, on a des signes de ponctuation associés aux mots (*walk?*) qui comptent pour des mots différents. Le tokenizer pourrait filtrer ces signes afin de ne produire que des mots "propres".

Cet exercice est fini, passons à l'activité 3.